# Business Location Analysis



<div style="text-align: right"> first drafted on 2016-07-11 </div>
<div style="text-align: right"> last updated on 2018-05-31 </div>

*It is a critical question to many business where to open and operate its offline spaces. Especially when a business is pursuing an expansion to places not so familiar to, researching and making a decision on where to locate can be a very difficult problem. I have had a similar concerns as a strategist, and came up with a simple alternative that helped me to make reasonable data-driven decision making.*

The logic is simple;
 
<br> 
<b>&nbsp;&nbsp;&nbsp;&nbsp;*'You want to do business from where your customers are.'*</b>


<br>
In order to find such a location, the following procedure has been taken from this analysis:

1. Define customers - Define sets of customers, which the business is targeting to. The purpose of business, marketing, and strategy all come in to play. If appripriate, apply STP to each set of customer.
2. Define sets - For each heterogeneous set of customers, allocation of brand or landmark that can represent characteristics of the set. It can be popular brand(shop), public places, or anything that can be searched on google map.
3. Find on google - Google web api provides geocode for your search query. Geocode for each keyword location can be retrieved.
4. Plot on map - Now that each location information is given, plot heatmap layer on google map with the help of package 'gmaps'. It can be done separately by each set of customers, or as a whole.

<br>
___
For example, *a restaurant franchise seeking an international expansion* into London, UK may take the following procedure:

1. Define customers
 - Curious customers seeking for new things to try with fellows, with mid-upper range income to spare on dinning experience.
2. Define sets
 - Restaurant(restaurant) 
 - Pioneers (university, waterstones, )
 - Thick wallet (starbucks, apple store, debenhams) 
 - Hang out (cinema, club)
3. Find on google
4. plot on map
___

The libraries used for this project are as follows;

- *re* for pattern & string process

- *gmaps* for Google maps API

- *urllib* for Google API and data crawling



In [1]:
from urllib.request import urlopen
import gmaps
import re

Define target city and retrieve geocode using google api

In [2]:
city = 'London'

city_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
key = "Your Key here"

In [3]:
url = urlopen(city_url+city+key)
tmp = url.readlines()
for i in range(len(tmp)):
    if '"location"' in str(tmp[i]):
        city_coord =(float(re.sub('[a-z\ ":\',\\\]*','',str(tmp[i+1]))), float(re.sub('[a-z\ ":\',\\\]*','',str(tmp[i+2]))))
str(city_coord)[1:-1]

'51.5073509, -0.1277583'

If you look into the data retrieved, it contains a number of location informations as follows;

In [4]:
tmp

[b'{\n',
 b'   "results" : [\n',
 b'      {\n',
 b'         "address_components" : [\n',
 b'            {\n',
 b'               "long_name" : "London",\n',
 b'               "short_name" : "London",\n',
 b'               "types" : [ "locality", "political" ]\n',
 b'            },\n',
 b'            {\n',
 b'               "long_name" : "London",\n',
 b'               "short_name" : "London",\n',
 b'               "types" : [ "postal_town" ]\n',
 b'            },\n',
 b'            {\n',
 b'               "long_name" : "Greater London",\n',
 b'               "short_name" : "Greater London",\n',
 b'               "types" : [ "administrative_area_level_2", "political" ]\n',
 b'            },\n',
 b'            {\n',
 b'               "long_name" : "England",\n',
 b'               "short_name" : "England",\n',
 b'               "types" : [ "administrative_area_level_1", "political" ]\n',
 b'            },\n',
 b'            {\n',
 b'               "long_name" : "United Kingdom",\n',
 b'   

___

Repeat the similar procedure for customer sets, within certain radius of the destination, London for instance.
Store coordinates in lists.

In [5]:
restaurant = ['restaurant']
pioneer = ['university', 'waterstones']
thick_wallet = ['starbucks', 'apple store', 'debenhams']
hang_out = ['cinema', 'club']

In [6]:
restaurant_loc = []
pioneer_loc = []
thick_wallet_loc = []
hang_out_loc = []

In [7]:
search_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='
search_range = '&radius=50000&name=' #radius in metre

In [8]:
def search(target, target_loc):
    for i in range(len(target)):
        url = (search_url+str(city_coord)[1:-1]+search_range+target[i]+key).replace(' ','')
        tmp = urlopen(url).readlines()
    for k in range(len(tmp)):
        if 'location' in str(tmp[k]):
            coordinate = (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+1])))), (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+2]))))
            target_loc.append(coordinate)

In [9]:
search(restaurant,restaurant_loc)
search(pioneer,pioneer_loc)
search(thick_wallet,thick_wallet_loc)
search(hang_out,hang_out_loc)


In [10]:
# gmap 기본 세팅 호출
gmaps.configure(api_key=key)

# 두꺼운 지갑들이 찾는 상권 분포도
restaurant_map = gmaps.figure(center=city_coord, zoom_level = 12)
restaurant_heatmap_layer = gmaps.heatmap_layer(restaurant_loc, point_radius = 30)
restaurant_map.add_layer(restaurant_heatmap_layer)
restaurant_map

Figure(layout=FigureLayout(height='420px'))

b. 마케팅 stp 별로 브랜드의 입지 좌표 확보

In [5]:
# 좌표 변수 생성
TW_coordinates = []
TS_coordinates = []
YS_coordinates = []
UP_coordinates = []
SUM_coordinates = []

# 구글 API에 요청하기 위한 주소 양식 생성
search_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='
search_mid = '&radius=50000&name='    # 넓은 범위 조사 시 radius 값 조정(단위: m)

# 구글 api를 활용하여 표적 브랜드의 좌표 확보 후 좌표변수에 할당
for i in range(len(thick_wallet)):
    url = (search_url+str(city_coord)[1:-1]+search_mid+thick_wallet[i]+key).replace(' ','')
    tmp = urlopen(url).readlines()
    for k in range(len(tmp)):
        if 'location' in str(tmp[k]):
            coordinate = (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+1])))), (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+2]))))
            TW_coordinates.append(coordinate)

for i in range(len(trend_setter)):
    url = (search_url+str(city_coord)[1:-1]+search_mid+trend_setter[i]+key).replace(' ','')
    tmp = urlopen(url).readlines()
    for k in range(len(tmp)):
        if 'location' in str(tmp[k]):
            coordinate = (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+1])))), (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+2]))))
            TS_coordinates.append(coordinate)
            
for i in range(len(young_student)):
    url = (search_url+str(city_coord)[1:-1]+search_mid+young_student[i]+key).replace(' ','')
    tmp = urlopen(url).readlines()
    for k in range(len(tmp)):
        if 'location' in str(tmp[k]):
            coordinate = (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+1])))), (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+2]))))
            YS_coordinates.append(coordinate)
            
for i in range(len(urban_population)):
    url = (search_url+str(city_coord)[1:-1]+search_mid+urban_population[i]+key).replace(' ','')
    tmp = urlopen(url).readlines()
    for k in range(len(tmp)):
        if 'location' in str(tmp[k]):
            coordinate = (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+1])))), (float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+2]))))
            UP_coordinates.append(coordinate)
            
SUM_coordinates = TW_coordinates + TS_coordinates + YS_coordinates + UP_coordinates

c. gmap 라이브러리를 활용하여 표적 도시 내 상권 분포 표시

In [6]:
TW_coordinates

[(36.3414912, 138.6367599),
 (36.2310581, 137.9645603),
 (36.6447253, 138.1881159),
 (36.23314939999999, 137.9554178),
 (36.1898041, 137.964001),
 (36.2399189, 137.9807163),
 (36.0303633, 138.1207761),
 (36.0269926, 138.0778445),
 (36.3928, 138.2577739),
 (35.8288026, 137.9689326),
 (36.233803, 137.9781522),
 (35.828887, 137.9689647),
 (36.5085755, 138.1340375),
 (36.2199852, 137.9768205),
 (36.3419297, 138.7397249),
 (36.2757014, 138.4640172),
 (36.0656415, 138.0523401),
 (36.2704636, 137.932108),
 (36.6434372, 138.1881635),
 (36.399297, 138.2443014),
 (36.23045820000001, 137.9649242),
 (36.64393889999999, 138.1889383),
 (36.231756, 137.9782059),
 (36.222151, 137.959867),
 (36.6368397, 138.1810196),
 (36.6454387, 138.1893021),
 (36.2333662, 137.9663294),
 (36.3955518, 138.2499167),
 (36.184307, 137.8929493),
 (36.644975, 138.188608),
 (36.2298159, 137.9680119),
 (36.65307, 138.1878368),
 (35.839374, 137.9599404)]

In [7]:
# gmap 기본 세팅 호출
gmaps.configure(api_key=key)

# 두꺼운 지갑들이 찾는 상권 분포도
TW_map = gmaps.figure(center=city_coord, zoom_level = 12)
TW_heatmap_layer = gmaps.heatmap_layer(TW_coordinates, point_radius = 30)
TW_map.add_layer(TW_heatmap_layer)
TW_map

Figure(layout=FigureLayout(height='420px'))

- 경제력이 있는 소비자들이 주로 찾는 상권은 상해 서남쪽에 주로 분포 (지하철 1호선, 2호선, 4호선, 7호선, 10호선 인근)

In [8]:
# 트렌더 세터들이 찾는 상권 분포도
TS_map = gmaps.figure(center=city_coord, zoom_level = 12)
TS_heatmap_layer = gmaps.heatmap_layer(TS_coordinates, point_radius = 30)
TS_map.add_layer(TS_heatmap_layer)
TS_map

Figure()

- 신문물에 관심이 많은 소비자들이 주로 찾는 상권은 상해 중심부의 북서부에 주로 분포 (지하철 1호선, 2호선, 4호선, 10호선 인근)

In [13]:
YS_map = gmaps.figure(center=city_coord, zoom_level = 12)
YS_heatmap_layer = gmaps.heatmap_layer(YS_coordinates, point_radius = 30)
YS_map.add_layer(YS_heatmap_layer)
YS_map

Figure()

- 상해시 내 대학들은 상해 서쪽과 북쪽 외곽에 분포 (지하철 1호선, 8호선, 10호선 인근)

In [12]:
UP_map = gmaps.figure(center=city_coord, zoom_level = 12)
UP_heatmap_layer = gmaps.heatmap_layer(UP_coordinates, point_radius = 30)
UP_map.add_layer(UP_heatmap_layer)
UP_map

Figure()

- 중상층 주거 계층의 분포는 상당부분 고르게 되어 있으나 주로 서북쪽에 분포가 높음

# 5. 종합

상기 4종의 상권 분포를 종합적으로 보았을 시 아래와 같은 결과를 도출할 수 있음

In [11]:
SUM_map = gmaps.figure(center=city_coord, zoom_level = 12)
SUM_heatmap_layer = gmaps.heatmap_layer(SUM_coordinates, point_radius = 30)
SUM_map.add_layer(SUM_heatmap_layer)
SUM_map

Figure()

- 종합적으로 보았을 때 A급 핵심 상권은 크게 "인민광장", "쉬자후이", "구베이" 3군데 정도로 압축 가능    

- 지하철 2호선, 7호선, 10호선 주변에 자원을 집중하는 것이 효율적이 될 수 있을 것으로 판단함

- 상해 동부의 푸동 교외 지역은 현재 상권이 개발 중으로, 현재는 약하지만 임대료가 저렴하고 향후 전망이 기대되어 전략적인 검토도 고려 필요